In [ ]:
import keys

In [ ]:
from openai import OpenAI
import json

client = OpenAI(api_key=keys.api)

# Define the tools for RAG and human handover with logic embedded in descriptions
tools = [
    {
        "type": 
        'function': {
            'name": "query_knowledge_base",
            "description": "Query the knowledge base to get answers to customer questions about specific topics and issues. This should be the first approach to solving customer issues before considering a handover to a human agent. Use this function to address the customer's question when sufficient information is provided or can be inferred.",
            "parameters": {
                "type": "object",
                "properties": {
                    "topic": {
                        "type": "string",
                        "description": "The last relevant topic of the question (e.g., 'billing', 'account', 'product features', 'technical issue') extracted from conversation history"
                    },
                    "issue": {
                        "type": "string",
                        "description": "A last relevant description of the customer's issue or question extracted from conversation history"
                    },
                },
                "required": ["topic", "issue"],
                "additionalProperties": False
            },
            "strict": True
        }
    },
    {
        "type": "function",
        "function": {
            "name": "handover_to_agent",
            "description": """Transfer the conversation to a human customer support agent. This function should ONLY be used in specific circumstances. Use this function ONLY IF: 
            1) the system didn't already provide an answer to the customer question using a knowledge base. OR the customer appears frustrated (all caps, exclamation marks, etc.)... one of those 2 conditions
            AND
            2) the customer requests human intervention""",
            "parameters": {
                "type": "object",
                "properties": {
                    "topic": {
                        "type": "string",
                        "description": "The last relevant topic of the question (e.g., 'billing', 'account', 'product features', 'technical issue') extracted from conversation history"
                    },
                    "issue": {
                        "type": "string",
                        "description": "A last relevant description of the customer's issue or question extracted from conversation history"
                    },
                },
                "required": ["topic", "issue"],
                "additionalProperties": False
            },
            "strict": True
        }
    }
]

# Generic system prompt without specific function logic
system_prompt = """You are a helpful customer support AI assistant. Your goal is to assist customers with their inquiries efficiently and effectively. You have access to tools that can help you best serve the customer. Use these tools appropriately based on their descriptions and the customer's needs."""

# Initialize variables to track conversation and function usage
conversation_history = []
rag_function_used = False
hho_function_used = False

def handle_customer_message(user_message):
    """Process a customer message and return the appropriate response"""
    global conversation_history, rag_function_used, hho_function_used
    
    # Add user message to conversation history
    conversation_history.append({"role": "user", "content": user_message})
    
    # Prepare messages for API call, including full conversation history
    messages = [{"role": "system", "content": system_prompt}]
    
    # Add all previous conversation messages
    messages.extend(conversation_history)
    
    # Make the API call
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tools
    )
    
    # Get assistant's response
    assistant_message = completion.choices[0].message
    
    # Store this message format for history
    history_message = {
        "role": "assistant",
        "content": assistant_message.content if assistant_message.content else None
    }
    
    # Process and display the result
    if hasattr(assistant_message, 'tool_calls') and assistant_message.tool_calls:
        # Add tool calls to the history message
        history_message["tool_calls"] = []
        
        for tool_call in assistant_message.tool_calls:
            # Add to history
            tool_call_dict = {
                "id": tool_call.id,
                "type": tool_call.type,
                "function": {
                    "name": tool_call.function.name,
                    "arguments": tool_call.function.arguments
                }
            }
            history_message["tool_calls"].append(tool_call_dict)
            
            print(f"INTERNAL PROCESSING: Function called - {tool_call.function.name}")
            print(f"Arguments: {tool_call.function.arguments}")
            
            # Execute the appropriate function based on what was called
            if tool_call.function.name == "query_knowledge_base":
                # Mark that RAG has been used
                rag_function_used = True
                
                # Parse arguments
                args = json.loads(tool_call.function.arguments)
                
                # Simulating RAG response
                result = f"Knowledge base result for topic: {args['topic']} and issue: {args['issue']}"
                
                # Create function response
                function_response = {
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": result
                }
                
                # Add function response to conversation history
                conversation_history.append(history_message)
                conversation_history.append(function_response)
                
                # Get final response from model
                final_messages = messages.copy()
                final_messages.append(assistant_message)
                final_messages.append(function_response)
                
                final_response = client.chat.completions.create(
                    model="gpt-4o",
                    messages=final_messages
                )
                
                # Add final response to conversation history
                final_message = {
                    "role": "assistant",
                    "content": final_response.choices[0].message.content
                }
                conversation_history.append(final_message)
                
                # print("\nFinal response to user:")
                print(final_response.choices[0].message.content)
                return final_response.choices[0].message.content
                
            elif tool_call.function.name == "handover_to_agent":
                
                hho_function_used = True
                # Here you would implement the actual agent handover functionality
                args = json.loads(tool_call.function.arguments)
                
                # Create function response
                function_response = {
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": "Human agent connected."
                }
                
                # Add messages to conversation history
                conversation_history.append(history_message)
                conversation_history.append(function_response)
                
                
                handover_message = "You're being connected to a human agent. Please wait a moment."
                
                # Add final handover message to history
                conversation_history.append({
                    "role": "assistant",
                    "content": handover_message
                })
                
                return handover_message
    else:
        # No function calls, just a direct response
        conversation_history.append(history_message)
        print("\nINTERNAL PROCESSING: Disambiguation")
        #print(assistant_message.content)
        return assistant_message.content

def main():
    """Main function to interact with the customer support system"""
    global conversation_history, rag_function_used, hho_function_used
    
    print("Customer Support System")
    print("Type 'exit' to end the conversation")
    print("----------------------------------")
    
    while True:
        user_input = input("\nCustomer: ")
        
        if user_input.lower() == 'exit':
            print("Conversation ended.")
            break
        
        print(f"User: {user_input}")
        response = handle_customer_message(user_input)
        print(f"AI: {response}")
        
        print("--------------Turn End------------")
        
        # Debug information
        # print("\n[Debug] RAG function used:", rag_function_used)
        # print("[Debug] HHO function used:", hho_function_used,"\n")

    # Print full conversation history
    print("\nFull conversation history:")
    for msg in conversation_history:
        if msg.get("role") == "user":
            print(f"Customer: {msg['content']}")
        elif msg.get("role") == "assistant" and msg.get("content"):
            print(f"AI: {msg['content']}")

if __name__ == "__main__":
    main()

In [ ]:

%load_ext autoreload
%autoreload 2


import helper as hlp

# Load all configurations
configs = hlp.load_all_configs()

# Access individual configs
planner_config = configs["planner"]
user_data = configs["user_data"]  # This is the parsed user_json as a dict
tools = configs["tools"]
content_items = configs["content"]["items"]

# Use the configuration data...
planner_mode = planner_config.get("mode", "fast")
combined_result = hlp.generate_schema_for_tools(tools)



In [ ]:


    # Process the tools with combined schema


print(planner_mode)


In [ ]:
combined_result

In [ ]:
toolz = [
    {
        "TaxToolz": {
            "region2": "US",
            "relevance_score": 85
        }
    },
    {
        "Untitled Tool": {
            "relevance_score": 25
        }
    }
]